# Subword Tokenizer

| **Tokenizer 방식** | **토큰 단위**                      | **vocab size** | **미등록 단어에 대한 가정**                                                                                  |
|---------------------|------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------|
| **사전 기반**       | 알려진 단어/형태소의 결합           | unlimited       | - 알려진 단어/형태소의 결합이라고 가정<br>- 필요한 형태소 분석 가능<br>- 사전에 등록되지 않은 단어는 UNK 처리 |
| **sub-word**        | 알려진 글자 및 sub-word            | fixed           | - 알려진 sub-words로 분해<br>- 예: appear → app + ear<br>- 자주 등장하는 단어를 제대로 인식 가능<br>- UNK의 개수 최소화 |

(memo)...

**WordPiece** : 구글에서 개발한 토크나이저, BERT 모델에서 사용
→ 확률기반
Pairs scores 높은 조합을 확률로 서브워드로 나눠 높은 빈도를 가진 확률로 뭉쳐있는 것들로 vocab

**BPE(Byte Pair Encoding)**: 빈도를 기반으로 문자 쌍을 반복적으로 병합

WordPiece, BPE는 단어의 경계를 고려하지만, SentencePiece는 고려하지 않는다.

**SentencePiece**: 구글에서 개발한 언어 독립적인 토크나이저
BPE, Unigram Languae Model 두 가지 알고리즘을 지원
유니코드 문자를 직접 처리, 공백도 하나의 심볼로 


In [13]:
# 네이버 영화 리뷰데이터

import urllib.request
import os

def get_file(filename, origin):
    cache_dir = os.path.expanduser('~/.torch/datasets')
    os.makedirs(cache_dir, exist_ok=True)
    filepath = os.path.join(cache_dir, filename)

    if not os.path.exists(filepath):
        print(f'Downloading data from {origin}')
        urllib.request.urlretrieve(origin, filepath)

    return filepath

In [14]:
ratings_train_path = get_file("ratings_train.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
ratings_test_path = get_file("ratings_test.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")

ratings_train_path, ratings_test_path

('C:\\Users\\Playdata/.torch/datasets\\ratings_train.txt',
 'C:\\Users\\Playdata/.torch/datasets\\ratings_test.txt')

In [15]:
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep="\t")
ratings_test_df = pd.read_csv(ratings_test_path, sep="\t")

display(ratings_train_df)
display(ratings_test_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [16]:
ratings_train_df.isna().sum()    # train : 5, test : 3 → 결측치 확인함

id          0
document    5
label       0
dtype: int64

In [17]:
ratings_train_df.shape, ratings_test_df.shape

((150000, 3), (50000, 3))

In [18]:
# 결측치 제거
ratings_train_df = ratings_train_df.dropna(how='any')
ratings_test_df = ratings_test_df.dropna(how='any')

ratings_train_df.shape, ratings_test_df.shape


((149995, 3), (49997, 3))

In [ ]:
# txt 파일 생성 - 학습 데이터

with open('naver_review.txt', 'w', encoding='UTF-8') as f:
    for doc in ratings_train_df['document'].values:
        f.write(doc + '\n')

### SentencePieceTokenizer *제일 유용*

In [ ]:
# !pip install sentencepiece

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 52.8 MB/s  0:00:00


(memo)...
sentencepiece 는 C++ 기반으로 만들어져있다.

In [22]:
import sentencepiece as stp

input = 'naver_review.txt'
vocab_size = 10000
model_prefix = 'naver_review'
cmd = f'--input={input} --model_prefix={model_prefix} --vocab_size={vocab_size}'

stp.SentencePieceTrainer.Train(cmd)

In [ ]:
sp = stp.SentencePieceProcessor()
sp.Load(f'{model_prefix}.model')       # 토크나이저 모델 로드

for doc in ratings_train_df['document'].values[:3]:
    print(doc)
    print(sp.encode_as_pieces(doc))    # 토큰화 → SentencePiece를 활용한 토큰화
    print(sp.encode_as_ids(doc))       # 시퀀싱 → 토큰화 후 정수 인코딩
    print()

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
[62, 877, 5, 31, 2019, 68, 1710]

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터', '보고', '▁초딩', '영화', '줄', '....', '오', '버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1634, 8, 4908, 159, 1460, 33, 264, 60, 173, 548, 410, 1224, 7396, 754, 440]

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보', '는것을', '추천', '한다']
[23, 369, 9781, 429, 3780, 143, 6266, 1945, 314]



In [ ]:
# 단어사전/ 모델의 어휘 크기(vocabulary size) 출력
sp.get_piece_size()    # 10000
# sp.GetPieceSize()    # 10000 → vocab size

10000

In [40]:
# 인코딩
text = ratings_test_df['document'][100]
tokens = sp.encode_as_pieces(text)       # 텍스트 → subword 단위 분할
id_tokens = sp.encode_as_ids(text)       # 텍스트 → subword 단위 분할 → 고유 ID 변환

# 토큰화 결과 확인
print(text)
print(tokens)
print(id_tokens)

# 디코딩
print("".join(tokens).replace("▁", " ").strip())
print(sp.decode_pieces(tokens))
print(sp.decode_ids(id_tokens))

걸작은 몇안되고 졸작들만 넘쳐난다.
['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
[1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.


### BertWordPieceTokenizer

In [ ]:
# !pip install tokenizers

In [34]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    lowercase=False,              # 소문자로 토큰 관리 X / 한국어는 대소문자가 없음
    strip_accents=False,          # 발음 강세문자 기호 제거 X / 한국어에는 자모 분리 가능성
)
vocab_size = 10000

tokenizer.train(
    files=['naver_review.txt'],  # 토크나이징 할 파일 경로
    vocab_size=vocab_size,       # 어휘 크기
    min_frequency=5,             # 최소 빈도 → 빈도가 5 이상인 토큰만 포함해서 단어 사전에 포함
    show_progress=True,          # 진행 표시
)

In [35]:
tokenizer.save_model('./', 'bert_word_piece_from_naver_review')

['./bert_word_piece_from_naver_review-vocab.txt']

In [41]:
text = ratings_test_df['document'][100]
encoded = tokenizer.encode(text)

print(encoded)
print(encoded.tokens)
print(encoded.ids)

print(tokenizer.decode(encoded.ids))

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['걸작', '##은', '몇', '##안되고', '졸작', '##들만', '넘쳐', '##난다', '.']
[2759, 1086, 445, 9499, 2589, 3799, 8337, 2430, 16]
걸작은 몇안되고 졸작들만 넘쳐난다.


##### 결과 비교 : SentencePieceTokenizer | BertWordPieceTokenizer

```python
 # SentencePieceTokenizer 토크나이저 결과

 걸작은 몇안되고 졸작들만 넘쳐난다.
 ['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
 [1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]
 걸작은 몇안되고 졸작들만 넘쳐난다.
 ```

```python
 # BertWordPieceTokenizer 토크나이저 결과

 ['걸작', '##은', '몇', '##안되고', '졸작', '##들만', '넘쳐', '##난다', '.']
 [2759, 1086, 445, 9499, 2589, 3799, 8337, 2430, 16]
 걸작은 몇안되고 졸작들만 넘쳐난다.
```

---

In [45]:
### 전처리 연습
docstring = """
1. 적절한 데이터셋을 찾거나 생성한다.
2. 적절한 전처리를 진행한다.
3. TfidfVectorizer를 활용해 문서를 백터화한다.
4. Cosine Similarity를 계산하여 입력된 문자열의 긍/부정 여부를 판단한다. 
"""